In [ ]:
import numpy as np

In [ ]:
def linear_probability(content_vector, remaining_array, user_array):
  #this returns a 1x num_users vector representing the probability for each user to select this content
  product = user_array@np.vstack((remaining_array,content_vector)).T
  return (product/product.sum(axis=1)[:,None])[:,-1]

In [ ]:
def exponential_probability(content_vector, remaining_array, user_array):
    exp = np.exp(user_array@content_vector)[:, None]
    #this returns a 1x num_users vector representing the probability for each user to select this content
    rem = user_array @ remaining_array.T

    return (exp/(exp + np.exp(rem.sum(axis=1)[:,None]))).flatten()

In [ ]:
def engagement_utility(content_vector, remaining_array, user_array, probability_function=linear_probability):
  #Args:
  # Content - The index of the vector representing the content in the producer_array
  # producer_array - a num_prodcuers x d array representing each contetn produced in the system
  # users_array - an num_users x d array represetning all_users and their preferences
  # probability - a function that maps the content and producer_arrary to probability for a piece of content being shown to each user ( 1x num_users vector)

  #get probabilty for each user
  probs = probability_function(content_vector, remaining_array, user_array)
  #get value for each user
  prods = user_array @ content_vector
  #since we define all users, we can just take an average
  #TODO: make a version of this for a distribution of users
  return sum(probs*prods)/num_users


In [ ]:
def is_best_response(content_vector, remaining_array, user_array):
  #determines if a given content vector is the best response to the other contents given the user array
  #True if the vector is a best response
  #TODO: this is probably dumb and there is a better way of doing it
  max_util = -1
  best_row = None
  for row in np.eye(user_dimension):
    util = engagement_utility(row, remaining_array, user_array)
    if util> max_util:
      best_row = row
      max_util = util
  return all(best_row == content_vector)

In [ ]:
def eval_position(producer_array, user_array):
  #returns True if all content vectors are the best response to one another given the user array
  for i, content in enumerate(producer_array):
    if not is_best_response(content, np.delete(producer_array, i, 0), user_array):
      return False
  return True

In [ ]:
def generate_uniform_user(dimension):
  #unifrom sampling form the simplex
  #reference : https://cs.stackexchange.com/questions/3227/uniform-sampling-from-a-simplex
  return np.diff([0] + sorted(np.random.uniform(size=dimension-1)) + [1])

In [ ]:
#establishing user variables

user_dimension = 4
num_users = 250

#each user is uniformly sampled from simplex
user_array = np.array([generate_uniform_user(user_dimension) for i in range(num_users)])
user_array

array([[2.78520093e-01, 4.37233165e-01, 1.36797447e-01, 1.47449294e-01],
       [2.74486114e-01, 6.10023321e-03, 3.20636533e-01, 3.98777120e-01],
       [7.06913213e-02, 2.84399873e-01, 5.08297533e-01, 1.36611273e-01],
       [2.99102441e-03, 7.81068425e-02, 3.21639645e-01, 5.97262488e-01],
       [8.44599255e-02, 8.10816682e-02, 6.51022873e-01, 1.83435533e-01],
       [2.33981932e-01, 3.82626889e-01, 2.50264910e-01, 1.33126270e-01],
       [2.38179182e-01, 1.39129902e-01, 8.24720520e-02, 5.40218864e-01],
       [2.03944966e-01, 1.38636093e-01, 7.48471422e-02, 5.82571799e-01],
       [3.45659981e-01, 1.97137883e-01, 2.37897560e-01, 2.19304576e-01],
       [1.58839389e-01, 1.20134492e-01, 6.51548708e-01, 6.94774109e-02],
       [5.78851233e-01, 8.29525205e-02, 7.10482302e-02, 2.67148016e-01],
       [1.25631173e-01, 3.12871654e-01, 1.07151117e-01, 4.54346056e-01],
       [2.29644001e-01, 4.93196173e-01, 2.38497159e-01, 3.86626676e-02],
       [7.76646662e-01, 9.19104368e-02, 7.41558818e

In [ ]:
num_producers = 2
#This generates all possible combinations of producers given the dimesnsion and number of producers
combinations = np.array(np.meshgrid(*[np.arange(user_dimension) for i in range(num_producers)])).T.reshape(-1, num_producers)
I = np.eye(user_dimension)
print(user_array)
print(user_array.sum(axis=0))
print('\n')
for combination in combinations:
  producer_array = np.zeros((num_producers, user_dimension))
  for i in range(num_producers):
    producer_array[i] = I[combination[i]]
    if eval_position(producer_array, user_array):
      print(producer_array)
      print('\n')


[[2.78520093e-01 4.37233165e-01 1.36797447e-01 1.47449294e-01]
 [2.74486114e-01 6.10023321e-03 3.20636533e-01 3.98777120e-01]
 [7.06913213e-02 2.84399873e-01 5.08297533e-01 1.36611273e-01]
 [2.99102441e-03 7.81068425e-02 3.21639645e-01 5.97262488e-01]
 [8.44599255e-02 8.10816682e-02 6.51022873e-01 1.83435533e-01]
 [2.33981932e-01 3.82626889e-01 2.50264910e-01 1.33126270e-01]
 [2.38179182e-01 1.39129902e-01 8.24720520e-02 5.40218864e-01]
 [2.03944966e-01 1.38636093e-01 7.48471422e-02 5.82571799e-01]
 [3.45659981e-01 1.97137883e-01 2.37897560e-01 2.19304576e-01]
 [1.58839389e-01 1.20134492e-01 6.51548708e-01 6.94774109e-02]
 [5.78851233e-01 8.29525205e-02 7.10482302e-02 2.67148016e-01]
 [1.25631173e-01 3.12871654e-01 1.07151117e-01 4.54346056e-01]
 [2.29644001e-01 4.93196173e-01 2.38497159e-01 3.86626676e-02]
 [7.76646662e-01 9.19104368e-02 7.41558818e-02 5.72870194e-02]
 [2.24297137e-01 2.38449545e-02 2.05917272e-01 5.45940637e-01]
 [9.21700145e-02 2.29621207e-01 4.78179593e-01 2.000291